## Loading an HCA matrix into seurat

This vignette illustrates requesting an expression matrix from the HCA matrix service and loading it into seurat.


First, install and import some dependencies:

In [140]:
library(httr)
install.packages("remotes")
install.packages("downloader")
library(downloader)
remotes::install_github("satijalab/seurat")
library(Seurat)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Skipping install of 'Seurat' from a github remote, the SHA1 (245d72b5) has not changed since last install.
  Use `force = TRUE` to force installation


Now, we're going to make some requests to describe what fields and values we can filter on when we're selecting our matrix.

In [141]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters")
content(r)

[[1]]
[1] "cell_suspension.provenance.document_id"

[[2]]
[1] "genes_detected"

[[3]]
[1] "specimen_from_organism.provenance.document_id"

[[4]]
[1] "specimen_from_organism.genus_species.ontology"

[[5]]
[1] "specimen_from_organism.genus_species.ontology_label"

[[6]]
[1] "donor_organism.human_specific.ethnicity.ontology"

[[7]]
[1] "donor_organism.human_specific.ethnicity.ontology_label"

[[8]]
[1] "donor_organism.diseases.ontology"

[[9]]
[1] "donor_organism.diseases.ontology_label"

[[10]]
[1] "donor_organism.development_stage.ontology"

[[11]]
[1] "donor_organism.development_stage.ontology_label"

[[12]]
[1] "derived_organ_ontology"

[[13]]
[1] "derived_organ_label"

[[14]]
[1] "derived_organ_part_ontology"

[[15]]
[1] "derived_organ_part_label"

[[16]]
[1] "library_preparation_protocol.provenance.document_id"

[[17]]
[1] "library_preparation_protocol.input_nucleic_acid_molecule.ontology"

[[18]]
[1] "library_preparation_protocol.input_nucleic_acid_molecule.ontology_label"

[[19]]
[1] "library_preparation_protocol.library_construction_method.ontology"

[[20]]
[1] "library_preparation_protocol.library_construction_method.ontology_label"

[[21]]
[1] "library_preparation_protocol.end_bias"

[[22]]
[1] "library_preparation_protocol.strand"

[[23]]
[1] "project.provenance.document_id"

[[24]]
[1] "project.project_core.project_short_name"

[[25]]
[1] "project.project_core.project_title"

[[26]]
[1] "analysis_protocol.provenance.document_id"

[[27]]
[1] "dss_bundle_fqid"

[[28]]
[1] "analysis_protocol.protocol_core.protocol_id"

[[29]]
[1] "analysis_working_group_approval_status"

That's the list of metadata fields we can filter on when requesting the matrix. We can describe any of them with further API calls:

In [142]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters/project.project_core.project_short_name")
content(r)

$cell_counts
$cell_counts$`Fetal/Maternal Interface`
[1] 1

$cell_counts$`Single cell RNAseq characterization of cell types produced over time in an in vitro model of human inhibitory interneuron differentiation.`
[1] 1733

$cell_counts$`Single cell transcriptome analysis of human pancreas`
[1] 2544


$field_description
[1] "A short name for the project."

$field_name
[1] "project.project_core.project_short_name"

$field_type
[1] "categorical"

In [143]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters/genes_detected")
content(r)

$field_description
[1] "Count of genes with a non-zero count."

$field_name
[1] "genes_detected"

$field_type
[1] "numeric"

$maximum
[1] 13108

$minimum
[1] 358

For categorical data, we see the number of cells associated with each category. For numeric, we see the range of value. If we want to request a matrix based on these metadata values, we can add them to the filter in the body of a POST request to the matrix service:

In [144]:
payload = list(
    filter =  list(
          op = "and", 
          value = list(
              list(op = "=", value = "Single cell transcriptome analysis of human pancreas",
                   field = "project.project_core.project_short_name"),
              list(op = ">=", value = 300,
                   field = "genes_detected")
    )),
    format = "csv"
)
r <- POST("https://matrix.data.humancellatlas.org/v1/matrix", body = payload, encode = "json")
response <- content(r)
print(response)

$eta
[1] ""

$matrix_url
[1] ""

$message
[1] "Job started."

$request_id
[1] "272a2c80-e953-406b-9715-41ecf8f5197c"

$status
[1] "In Progress"



That call responds right away and tells us that the matrix is being prepared. We can use the request_id to wait until the matrix is done.

In [145]:
request_id <- response["request_id"]
status <- response["status"]
message(status)
while (status != "Complete") 
{
    url = paste("https://matrix.data.humancellatlas.org/v1/matrix/", request_id, sep = "")
    r <- GET(url)
    response <- content(r)
    status = response["status"]
    message(status)
    Sys.sleep(15)
}
print(response)

In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
Complete


$eta
[1] ""

$matrix_url
[1] "https://s3.amazonaws.com/dcp-matrix-service-results-prod/272a2c80-e953-406b-9715-41ecf8f5197c.csv.zip"

$message
[1] "Request 272a2c80-e953-406b-9715-41ecf8f5197c has successfully completed. The resultant expression matrix is available for download at https://s3.amazonaws.com/dcp-matrix-service-results-prod/272a2c80-e953-406b-9715-41ecf8f5197c.csv.zip"

$request_id
[1] "272a2c80-e953-406b-9715-41ecf8f5197c"

$status
[1] "Complete"



Now, that the matrix is ready, we can download it. The file we download is a zip archive that contains a readme and a csv-formatted matrix. Other formats (loom, mtx) can be specified in the matrix request.

In [146]:
matrix_file_url = unlist(response["matrix_url"])

download.file(url=matrix_file_url,
              destfile='matrix.zip', method='curl')
unzip("matrix.zip", exdir = "./")
file.show('./csv_readme.md')







Finally, we load the expression matrix into a seurat object.

In [148]:
data_dir = paste("./", request_id, ".csv/", sep = "")
list.files(data_dir)
raw_counts<-read.table(file=paste0(data_dir,"expression.csv"),sep=",")
mydata <- CreateSeuratObject(counts = raw_counts, project = "Single cell transcriptome analysis of human pancreas", assay = "Smart-seq-2")
mydata



[1] "cells.csv"      "expression.csv" "genes.csv"

Warning message in storage.mode(from) <- "double":
“NAs introduced by coercion”

An object of class Seurat 
2545 features across 63926 samples within 1 assay 
Active assay: Smart-seq-2 (2545 features)